# Database Examples 

Examples for the new database schema and how to use it.

In [1]:
! pip install psycopg2-binary python-dotenv


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# jupyter notebook auto reload
%cd ..
%load_ext autoreload
%autoreload 2

/home/andre/rts


## Setup

Utility functions for setting up the database.

In [5]:
import os
import psycopg2
from dotenv import load_dotenv
from rts.db.utils import execute_read_query, execute_write_query

load_dotenv()

True

## Tables

Creating the necessary database tables

In [6]:
execute_write_query("DROP TABLE IF EXISTS features;")
execute_write_query("DROP TABLE IF EXISTS media;")

In [7]:
# postgres table definitions

_table_media = """
    CREATE TABLE IF NOT EXISTS media (
        media_id SERIAL PRIMARY KEY,
        media_path VARCHAR(500) UNIQUE,
        original_path VARCHAR(500) NOT NULL,
        created_at TIMESTAMP DEFAULT NOW(),
        media_type VARCHAR(50) NOT NULL,
        sub_type VARCHAR(50) NOT NULL,
        size INTEGER NOT NULL,
        archive_name VARCHAR (50) NOT NULL,
        archive_id VARCHAR (50) NOT NULL,
        metadata JSONB NOT NULL
    );
"""

_table_features = """
    CREATE TABLE IF NOT EXISTS features (
        feature_id SERIAL PRIMARY KEY,
        feature_type VARCHAR(50) NOT NULL,
        version VARCHAR(20) NOT NULL,
        created_at TIMESTAMP DEFAULT NOW(),
        model_name VARCHAR(200) NOT NULL,
        model_params JSONB NOT NULL,
        data JSONB NOT NULL,

        embedding_size INTEGER,
        embedding_1024 vector (1024),
        embedding_2048 vector (2048),

        media_id INTEGER,

        CONSTRAINT FK_features_media_id FOREIGN KEY (media_id) 
            REFERENCES media (media_id)
    );
"""

execute_write_query(_table_media)
execute_write_query(_table_features)

## Fill tables with sample data

In [8]:
# create sample data for the tables, each media element can have multiple features

swiss_cities = ["Zurich", "Geneva", "Basel", "Lausanne", "Bern", "Winterthur", "Lucerne", "St. Gallen", "Lugano", "Biel/Bienne"]
years = [2015, 2016, 2017, 2018, 2019, 2020]

for i in range(1, 21):
    _table_media_sample = f"""
        INSERT INTO media (media_path, original_path, media_type, sub_type, size, archive_name, archive_id, metadata)
        VALUES 
            ('/path/to/media{i}', '/path/to/original{i}', 'image', 'jpg', 500, 'archive{i}', 'archive_id{i}', '{{"key{i}": "value{i}", "city": "{swiss_cities[(i-1) % len(swiss_cities)]}", "year": {years[(i-1) % len(years)]}}}')
        RETURNING media_id;
    """
    execute_write_query(_table_media_sample)


In [9]:
_query = """SELECT * FROM media LIMIT 2;"""
execute_read_query(_query)

[(1,
  '/path/to/media1',
  '/path/to/original1',
  datetime.datetime(2023, 4, 28, 13, 0, 0, 700233),
  'image',
  'jpg',
  500,
  'archive1',
  'archive_id1',
  {'city': 'Zurich', 'key1': 'value1', 'year': 2015}),
 (2,
  '/path/to/media2',
  '/path/to/original2',
  datetime.datetime(2023, 4, 28, 13, 0, 0, 738503),
  'image',
  'jpg',
  500,
  'archive2',
  'archive_id2',
  {'city': 'Geneva', 'key2': 'value2', 'year': 2016})]

In [10]:
# insert some samples with vectors
import numpy as np
import json

feature_types = ["pose", "face", "object", "ner"]
color_meta = ["red", "green", "blue", "yellow", "black", "white", "grey", "orange", "purple", "pink"]

for i in range(1, 11):
    vector_1024 = np.random.rand(1024).tolist()
    vector_2048 = np.random.rand(2048).tolist()
    ner_tags = json.dumps([("person", "Ueli Steck", 1, 9), ("city", "geneva", 10, 16), ("city", "zurich", 17, 23), ("city", "bern", 24, 28), ("city", "basel", 29, 34), ("city", "winterthur", 35, 46), ("city", "lucerne", 47, 54), ("city", "st. gallen", 55, 65), ("city", "lugano", 66, 72), ("city", "biel/bienne", 73, 85)])

    _table_features_sample_vectors = f"""
        INSERT INTO features (feature_type, version, model_name, model_params, data, media_id, embedding_size, embedding_1024)
        VALUES
            ('{feature_types[i % len(feature_types)]}', 'v1', 'resnet50', '{{"param1": "{i}"}}', '{{"color": "{color_meta[i % len(color_meta)] }", "data1": "{i}", "ner": {ner_tags} }}', {i}, 1024, ARRAY[{','.join([str(x) for x in vector_1024])}])
        RETURNING feature_id;
    """
    # print(_table_features_sample_vectors)
    execute_write_query(_table_features_sample_vectors)

    # At the moment we are only creating size 1024 vectors, for the sake of the next example queries to work, there can be only a single vector set per feature
    # _table_features_sample_vectors = f"""
    #     INSERT INTO features (feature_type, version, model_name, model_params, data, media_id, embedding_size, embedding_2048)
    #     VALUES
    #         ('image', 'v1', 'resnet50', '{{"param1": "{i}"}}', '{{"data1": "{i}"}}', {i}, 2048, ARRAY[{','.join([str(x) for x in vector_2048])}])
    #     RETURNING feature_id;
    # """

    # execute_write_query(_table_features_sample_vectors)


## Queries

### Scenario 1

We have a media object and we want to find the 5 most similar media objects.

In [11]:
_query = """
    WITH target_embedding AS (
    SELECT
        media_id,
        CASE
            WHEN embedding_size = 1024 THEN embedding_1024
            WHEN embedding_size = 2048 THEN embedding_2048
            ELSE NULL
        END AS embedding_vector
    FROM 
        features
    WHERE 
        media_id = 5
    )

    SELECT
    f.media_id,
    (target.embedding_vector <-> 
        CASE
        WHEN f.embedding_size = 1024 THEN f.embedding_1024
        WHEN f.embedding_size = 2048 THEN f.embedding_2048
        ELSE NULL
        END
    ) AS distance
    FROM
    features f,
    target_embedding target
    WHERE
    f.media_id != target.media_id
    ORDER BY
    distance ASC
    LIMIT 5;
"""

execute_read_query(_query)

[(10, 12.7321838886941),
 (3, 12.8817617441207),
 (7, 12.9845741023454),
 (2, 13.0395603121536),
 (9, 13.0573006105726)]

### Scenario 2

Find all media objects for Zurich.

In [12]:
# get all media objects that have city: "Zurich" (queried from the jsonb metadata field)
_query = """
    SELECT * FROM media WHERE metadata->>'city' = 'Zurich';
"""
execute_read_query(_query)

[(1,
  '/path/to/media1',
  '/path/to/original1',
  datetime.datetime(2023, 4, 28, 13, 0, 0, 700233),
  'image',
  'jpg',
  500,
  'archive1',
  'archive_id1',
  {'city': 'Zurich', 'key1': 'value1', 'year': 2015}),
 (11,
  '/path/to/media11',
  '/path/to/original11',
  datetime.datetime(2023, 4, 28, 13, 0, 1, 178896),
  'image',
  'jpg',
  500,
  'archive11',
  'archive_id11',
  {'city': 'Zurich', 'year': 2019, 'key11': 'value11'})]

### Scenario 3

Get all images from Geneva

In [13]:
_query = """
    SELECT * FROM media WHERE metadata->>'city' = 'Geneva' AND media_type = 'image';
"""
execute_read_query(_query)


[(2,
  '/path/to/media2',
  '/path/to/original2',
  datetime.datetime(2023, 4, 28, 13, 0, 0, 738503),
  'image',
  'jpg',
  500,
  'archive2',
  'archive_id2',
  {'city': 'Geneva', 'key2': 'value2', 'year': 2016}),
 (12,
  '/path/to/media12',
  '/path/to/original12',
  datetime.datetime(2023, 4, 28, 13, 0, 1, 229458),
  'image',
  'jpg',
  500,
  'archive12',
  'archive_id12',
  {'city': 'Geneva', 'year': 2020, 'key12': 'value12'})]

### Scenario 4

Fulltext string matching on jsonb fields

In [14]:
# query for cities that start with b
_query = """
    SELECT * FROM media WHERE metadata->>'city' LIKE 'B%';
"""
print([x[9]['city'] for x in execute_read_query(_query)])

_query = """
    SELECT * FROM media WHERE metadata->>'city' LIKE '%Gall%';
"""
[x[9]['city'] for x in execute_read_query(_query)]



['Basel', 'Bern', 'Biel/Bienne', 'Basel', 'Bern', 'Biel/Bienne']


['St. Gallen', 'St. Gallen']

### Scenario 5

Similarity to a computed vector. Example: we have a video camera installed and a user poses like a tennis player and we find tennis matches.

In [15]:
vector_1024 = ",".join([str(x) for x in np.random.rand(1024).tolist()])  # feature that would be creating by the pose detection algorithm

# query the feature table for similar vectors
_query = f"""
    SELECT
    f.media_id,
    ('[{vector_1024}]' <-> 
        CASE
        WHEN f.embedding_size = 1024 THEN f.embedding_1024
        WHEN f.embedding_size = 2048 THEN f.embedding_2048
        ELSE NULL
        END
    ) AS distance
    FROM
    features f
    WHERE
    f.feature_type = 'pose'
    ORDER BY
    distance ASC
    LIMIT 5;
"""

execute_read_query(_query)


[(4, 12.7834132878445), (8, 13.2849294315309)]

### Scenario 6

Find people or locations with ner tags

In [16]:
person = "Ueli Steck"

_query = f"""
    SELECT media_id, data FROM features WHERE data->>'ner' LIKE '%{person}%';
"""

execute_read_query(_query)


[(1,
  {'ner': [['person', 'Ueli Steck', 1, 9],
    ['city', 'geneva', 10, 16],
    ['city', 'zurich', 17, 23],
    ['city', 'bern', 24, 28],
    ['city', 'basel', 29, 34],
    ['city', 'winterthur', 35, 46],
    ['city', 'lucerne', 47, 54],
    ['city', 'st. gallen', 55, 65],
    ['city', 'lugano', 66, 72],
    ['city', 'biel/bienne', 73, 85]],
   'color': 'green',
   'data1': '1'}),
 (2,
  {'ner': [['person', 'Ueli Steck', 1, 9],
    ['city', 'geneva', 10, 16],
    ['city', 'zurich', 17, 23],
    ['city', 'bern', 24, 28],
    ['city', 'basel', 29, 34],
    ['city', 'winterthur', 35, 46],
    ['city', 'lucerne', 47, 54],
    ['city', 'st. gallen', 55, 65],
    ['city', 'lugano', 66, 72],
    ['city', 'biel/bienne', 73, 85]],
   'color': 'blue',
   'data1': '2'}),
 (3,
  {'ner': [['person', 'Ueli Steck', 1, 9],
    ['city', 'geneva', 10, 16],
    ['city', 'zurich', 17, 23],
    ['city', 'bern', 24, 28],
    ['city', 'basel', 29, 34],
    ['city', 'winterthur', 35, 46],
    ['city', 'lu

### Scenario 7

Find aribitrary features by metadata (here we use the simple field color as an example)

In [17]:
color = "red"

_query = f"""
    SELECT media_id, data FROM features WHERE data->>'color' = '{color}';
"""

execute_read_query(_query)


[(10,
  {'ner': [['person', 'Ueli Steck', 1, 9],
    ['city', 'geneva', 10, 16],
    ['city', 'zurich', 17, 23],
    ['city', 'bern', 24, 28],
    ['city', 'basel', 29, 34],
    ['city', 'winterthur', 35, 46],
    ['city', 'lucerne', 47, 54],
    ['city', 'st. gallen', 55, 65],
    ['city', 'lugano', 66, 72],
    ['city', 'biel/bienne', 73, 85]],
   'color': 'red',
   'data1': '10'})]

# Ingest the clips to the media table

In [18]:
REMOTE_RTS_DATA = "/media/sinergia/RTS/"
REMOTE_VIDEOS = '/mnt/rts/'

LOCAL_RTS_DATA = "/media/data/rts/"
METADATA = LOCAL_RTS_DATA + 'metadata'
LOCAL_VIDEOS = LOCAL_RTS_DATA + 'archive'

AIBOX = LOCAL_RTS_DATA + 'aibox-vectors'

OUTDIR = 'data'

In [19]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple
from pathlib import Path
import shutil
from supabase import create_client, Client

In [20]:
# LOCAL imports
import rts
import rts.pipeline
import rts.utils
import rts.io.media
import rts.features.audio
import rts.features.text

LOG = rts.utils.get_logger()

/home/andre/rts/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
supabase: Client = create_client(
    os.getenv("SUPABASE_HOST"), 
    os.getenv("SUPABASE_KEY")
)

In [40]:
name = "rts"
res = supabase.storage.create_bucket(name)

StorageException: {'statusCode': 400, 'error': 'invalid signature', 'message': 'invalid signature'}

In [21]:
df = rts.utils.dataframe_from_hdf5(LOCAL_RTS_DATA + '/metadata', 'rts_aivectors')
df.shape

(3177, 22)

In [22]:
# get all the clips from the archive and put them into supabase s3. At the same time create the database entries on the media table

error_count = 0
no_clips = 0

for i, row in df.iterrows():

    path = os.path.join(row.mediaFolderPath.replace(REMOTE_VIDEOS, LOCAL_VIDEOS + '/'), 'clips', 'videos')

    # get all files in the folder
    try:
        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        no_clips += len(files)
    except FileNotFoundError:
        error_count += 1
        continue

error_count, no_clips

(2248, 5012)

In [24]:
path, row['mediaFolderPath'], files

('/media/data/rts/archive/1/2/0/ZB187021/clips/videos',
 '/mnt/rts/1/2/0/ZB187021',
 ['ZB023013-L010.mp4',
  'ZB023013-L009.mp4',
  'ZB023013-L000.mp4',
  'ZB023013-L023.mp4',
  'ZB023013-L017.mp4',
  'ZB023013-L024.mp4',
  'ZB023013-L012.mp4',
  'ZB023013-L018.mp4',
  'ZB023013-L014.mp4',
  'ZB023013-L015.mp4',
  'ZB023013-L032.mp4',
  'ZB023013-L004.mp4',
  'ZB023013-L033.mp4',
  'ZB023013-L001.mp4',
  'ZB023013-L011.mp4',
  'ZB023013-L028.mp4',
  'ZB023013-L025.mp4',
  'ZB023013-L031.mp4',
  'ZB023013-L022.mp4',
  'ZB023013-L013.mp4',
  'ZB023013-L021.mp4',
  'ZB023013-L016.mp4',
  'ZB023013-L034.mp4',
  'ZB023013-L019.mp4',
  'ZB023013-L005.mp4',
  'ZB023013-L002.mp4',
  'ZB023013-L020.mp4',
  'ZB023013-L027.mp4',
  'ZB023013-L029.mp4',
  'ZB023013-L030.mp4',
  'ZB023013-L006.mp4',
  'ZB023013-L008.mp4',
  'ZB023013-L007.mp4',
  'ZB023013-L003.mp4',
  'ZB023013-L026.mp4'])

x

'http://localhost:8000'